In [1]:
import os
import tensorflow as tf
import sys
import math
import cv2
import imutils
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras.models import model_from_json
from tensorflow.keras.backend import resize_images
from tensorflow.keras.models import load_model
print(tf.__version__)

1.15.0


In [3]:
model_all = load_model('Model/trial_all_class_weights.h5')
model_correct = load_model('Model/trial_correct.h5')
model_curve = load_model('Model/trial_curve.h5')
model_fingers_up = load_model('Model/trial_fingers_up.h5')
model_fist = load_model('Model/trial_fist.h5')
model_lkxy = load_model('Model/trial_lkxy.h5')
model_point = load_model('Model/trial_point.h5')
print("Model loaded!")
branch = {}
branch[0] = model_fist
branch[1] = model_fingers_up
branch[2] = model_curve
branch[3] = model_fingers_up
branch[4] = model_fist
branch[5] = model_correct
branch[6] = model_point
branch[7] = model_point
branch[8] = model_fingers_up
branch[10] = model_lkxy
branch[11] = model_lkxy
branch[12] = model_fist
branch[13] = model_fist
branch[14] = model_curve
branch[15] = model_correct
branch[16] = model_correct
branch[17] = model_fingers_up
branch[18] = model_fist
branch[19] = model_point
branch[20] = model_fingers_up
branch[21] = model_correct
branch[22] = model_correct
branch[23] = model_lkxy
branch[24] = model_lkxy
print("Branch models added")

Model loaded!
Branch models added


In [6]:
def predict(clone):
    crop_img = clone[100:350,50:300]
    resized_img = cv2.resize(crop_img,(100,100))
    
    # Required Preprocessing
    img = cv2.cvtColor(resized_img ,cv2.COLOR_BGR2YUV)
    img[:,:,0] = cv2.equalizeHist(img[:,:,0])
    img_hist = cv2.cvtColor(img, cv2.COLOR_YUV2RGB)
    denoised = cv2.bilateralFilter(img_hist, 7, 100, 100) 
#     v = np.median(denoised)
#     lower = int(max(0, (1.0 - 0.33) * v))
#     upper = int(min(255, (1.0 + 0.33) * v))
#     img_blur_canny = cv2.Canny(denoised,lower,upper)

    x_test = np.asarray(denoised)
    
    y_pred = model_all.predict(x_test.reshape((1,100,100,3)), batch_size=1)
    max_pred = np.max(y_pred)
    predicted_char = ""
    if(max_pred>0.5):
        predicted_char = chr(ord('A') + np.argmax(y_pred))
    return predicted_char

In [10]:
videoCaptureObject = cv2.VideoCapture(0)
ret = True   
size = (640, 480)
# out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, size) 
start_point = (50,100)
end_point = (300, 350) 
predict_string = "Word:"
while(ret):
    ret,frame = videoCaptureObject.read()
    if ret == True:
        clone = frame.copy() 
        
        predictKey = cv2.waitKey(1) & 0xFF
        if predictKey == ord('p'):
            c = predict(clone)
            predict_string += c
        elif predictKey == ord('e'):
            predict_string = predict_string[:-1]
        elif predictKey == ord('n'):
            predict_string = "Word:"
        elif predictKey == ord(' '):
            predict_string += " "
                 
        display_text = "press p for prediction"
        frame = cv2.rectangle(frame, start_point, end_point, color=(0,0,255), thickness=2)
        frame = cv2.flip(frame,1)
        cv2.putText(frame, display_text, (285, 85), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255), 3)
        cv2.putText(frame, predict_string, (320, 385), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255), 3)
        cv2.imshow('ASL Alphabet Classification', frame)        
        
#         out.write(frame)
        pressedKey = cv2.waitKey(1) & 0xFF
        if(pressedKey == 27):  # esc key
            videoCaptureObject.release()
            out.release()
            cv2.destroyAllWindows()
    else:
        print("Prediction Done")


Prediction Done
